### RETRIEVAL AUGMENTED GENERATION

<img src="../assets/rag_macro.png" alt="drawing" width="600"/>

#### Initialisation du model
Nous réutilisons le même modèle que pour le chat bot simple

In [18]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

#### Preparation du Prompt Template

Nous procédons à deux évolutions dans le `prompt template`:
* Le message System évolue pour préciser ce qui est attendu du modèle
* Un nouveau placeholder est créé pour y injecter les documents pertinents à fournir au modèle pour générer la réponse


In [19]:
from langchain_core.prompts import (
    ChatPromptTemplate,  
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate, 
    MessagesPlaceholder
)

prompt_template = ChatPromptTemplate.from_messages(
    [
       SystemMessagePromptTemplate.from_template( 
           """You are an assistant for question-answering tasks. 
            Use the following pieces of retrieved context to answer the question.
            If you don't know the answer, just say that you don't know. 
            Use three sentences maximum and keep the answer concise."""),
       MessagesPlaceholder(variable_name="history"),
       SystemMessagePromptTemplate.from_template("{context}"),
       HumanMessagePromptTemplate.from_template("{question}")
    ]
)

print(prompt_template)

input_variables=['context', 'history', 'question'] input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant for question-answering tasks. \n            Use the following pieces of retrieved context to answer the question.\n            If you don't know the answer, just say that you don't know. \n            Use three sentences maximum and keep the answer concise.")), MessagesPlaceholder(variable_name='history'), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='{context}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]


#### Initialisation de la mémoire
Pour la mémoire nous utiliserons cette fois-ci une instance Redis locale

In [20]:
from langchain_community.chat_message_histories import RedisChatMessageHistory
from os import getenv
history = RedisChatMessageHistory("genai_in_action_simple_rag", url=getenv("REDIS_URL"))

#### Ingestion du document

<img src="../assets/rag_load_split_embed.png" alt="drawing" width="900"/>

Pour l'étape `Load`, nous utilisons un des nombreux `Loader` natif de Langchain, ici celui traitant les fichiers `txt`

In [21]:
from langchain_community.document_loaders import PyPDFLoader

# LOAD
loader = PyPDFLoader("../__docs__/NO-GRH-010 Gestion des déplacements et défraiement_1.0.pdf")
docs = loader.load()
print(docs[0].page_content[:250])
print(docs[0].metadata)

 GESTION DES D ÉPLACEMENTS ET DÉFRAIEMENT  
 
 
Date de création  : Novembre 2022  Page 1 sur 2 NO-GRH -010_1.0  
Ce document permet de présenter la gestion des frais de déplacement et de défraiement au sein 
de Younup.  
L’approche globale de l’entr
{'source': '../__docs__/NO-GRH-010 Gestion des déplacements et défraiement_1.0.pdf', 'page': 0}


Pour le `Split`, le document est découpé en *chunk* de 1000 caractères avec un recoupement (*overlap*) entre eux de 200 caractères.

In [22]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

# SPLIT
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

splits

[Document(page_content='GESTION DES D ÉPLACEMENTS ET DÉFRAIEMENT  \n \n \nDate de création  : Novembre 2022  Page 1 sur 2 NO-GRH -010_1.0  \nCe document permet de présenter la gestion des frais de déplacement et de défraiement au sein \nde Younup.  \nL’approche globale de l’entreprise est d’avoir une politique homogène, structurée et en accord avec \nsa démarche RSE . \nDans ce cadre , plusieurs rubriques  sont à prendre en compte  : le transport, l e logement et la \nrestauration.   \nEn règle générale, les collabo rateurs de  Younup doivent  privilégier les moyens de transport doux  \net en particulier  pour ces déplacements ponctuels, le train.  \n \n \nDéplacement s demandé s par le client  \nProcess à réaliser et règle s pour le consultant  : \n- Prévenir  le commercial qui suit la prestation en cours ou son manager . \n- S’assurer d’avoir un ordre de mission  couvrant ce déplacement (essentiel pour des raisons \nd’assurance ) avant le déplacement.', metadata={'source': '../__docs

Enfin pour la partie `Embed`et `Store`, nous faisons appel au modèle de LLM d'embedding d'penAi (`ada`) pour réaliser les embeddings et les stockons dans le vector database du [Redis local](http://localhost:8001/redis-stack/browser)

In [23]:
from langchain_community.vectorstores.redis import Redis
from langchain_openai import OpenAIEmbeddings

vectorstore = Redis.from_documents(
    splits,
    OpenAIEmbeddings(),
    redis_url=getenv("REDIS_URL"),
    index_name="genai_in_action_simple_rag",
)

#### Création de la chaîne OpenAI

La chaîne finale se complexifie pour être une composition de plusieurs chaînes:

La chaîne principale est une chaîne séquentielle qui va :
1. Traiter les inputs
2. Passer ces inputs traités au prompt template
3. Passer le prompt termplate variabilisé au model
4. Passer la réponse à un parser pour en extraire simplement la réponse au format texte

Le premier élément de la chaîne est elle-même une chaîne qui va traiter les inputs en parrallèle:
* `question` et `history` sont simplement passé tels quels
* `context` est produit via une 3ème chaîne séquentielle qui va à partir de la question de l'utilistaeur retourné les bouts de document les plus pertinents du vector store

In [24]:
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    # Runnable parallèles
    {
        "context": itemgetter("question") | vectorstore.as_retriever() | format_docs, 
        "question": itemgetter("question"), 
        "history": itemgetter("history")
    }
    # Runnable séquentiels
    | prompt_template
    | model
    | StrOutputParser()
)


On peut maintenant poser des questions sur le document

In [25]:
question1 ="Quelle est la politique de déplacement de Younup ?"

response1 = rag_chain.invoke(
    {
        "question": question1,
        "history": history.messages
    }
)
response1

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


'La politique de déplacement de Younup privilégie les moyens de transport doux, avec une préférence particulière pour le train pour des déplacements ponctuels. Les collaborateurs doivent avoir un ordre de mission couvrant le déplacement pour des raisons d’assurance. Les modalités de déplacement (transport, logement, repas) sont définies entre Younup et le client, et si le client demande à Younup de gérer le déplacement, le service RH doit être informé pour la gestion des modalités.'

#### Bonus: Streamer la réponse

In [26]:
question2 ="Quelles sont les personnes à contacter ?"
response2 = ""

for chunk in rag_chain.stream(
    {
        "question": question2,
        "history": history.messages
    }):
    print(chunk)
    response2+=chunk

print(f"Réponse complète: {response2}")



APIConnectionError: Connection error.